Tensorflow and Numpy implementations of [Bhaskara I's sine approximation formula](https://en.wikipedia.org/wiki/Bhaskara_I%27s_sine_approximation_formula), extended to approximate sin/cos everywhere instead of just at the origin. This approximates sin and cosine everywhere to within a tolerance of 0.0017. Profiling this code suggests that it's significantly slower than the built in sin/cos implementations in Numpy and TensorFlow, but it might be a useful starting point for a custom op.

In [0]:
import time
import numpy as np
import matplotlib.pyplot as plt

In [0]:
x = np.float32(np.linspace(-np.pi*4, np.pi*4, 10001))

def cosb(x):
  x_mod_sq = 0.25 * (np.pi - np.abs(np.pi - 2*(x % np.pi)))**2
  x_sign = 1 - 2 * (((np.pi/2 + x) % (2*np.pi) > np.pi))
  x_cos = x_sign * (np.pi**2 - 4*x_mod_sq)/(np.pi**2 + x_mod_sq)
  return x_cos

def sinb(x):
  return cosb(x - np.pi/2)

cosx = np.cos(x)
sinx = np.sin(x)
cosbx = cosb(x)
sinbx = sinb(x)

def test(y1, y2, t = 0.0017):
  plt.plot(x, y1, linewidth=5)
  plt.plot(x, y2)
  err = np.max(np.abs(y1 - y2))
  print(err)
  assert err < t

test(cosx, cosbx)
test(sinx, sinbx)

runtimes = []
for _ in range(100):
  t0 = time.time()
  y = cosb(x)
  y = sinb(x)
  t1 = time.time()
  runtimes.append(t1 - t0)
time_approx = 1000 * np.median(runtimes)

runtimes = []
for _ in range(100):
  t0 = time.time()
  y = np.cos(x)
  y = np.sin(x)
  t1 = time.time()
  runtimes.append(t1 - t0)
time_default = 1000 * np.median(runtimes)
print(f'{time_default:0.5f} -> {time_approx:0.5f} | {time_default / time_approx:0.5f}x')

In [0]:
import tensorflow as tf

x = tf.convert_to_tensor(np.float32(np.linspace(-np.pi*4, np.pi*4, 10001)))

@tf.function
@tf.custom_gradient
def cosb(x):
  x_mod_sq = 0.25 * (np.pi - tf.math.abs(np.pi - 2*(x % np.pi)))**2
  x_sign = 1 - 2 * tf.cast(((np.pi/2 + x) % (2*np.pi) > np.pi), x.dtype)
  x_cos = x_sign * (np.pi**2 - 4*x_mod_sq)/(np.pi**2 + x_mod_sq)
  # TF's autodiff of `y` is wrong where x_sign switches over, because instead
  # of returning a valid subgradient there TF returns 0. This can be fixed by
  # using trig identities and a custom grad. Note that this gradient is an
  # accurate approximation to the gradient of non-approximate cos(), but is a
  # less accurate approximation to the value returned by this function.
  def grad(dy):
    return -cosb(x - np.pi/2) * dy
  return x_cos, grad

@tf.function
def sinb(x):
  return cosb(x - np.pi/2)

with tf.GradientTape(persistent=True) as tape:
  tape.watch(x)
  cosx = tf.math.cos(x)
  sinx = tf.math.sin(x)
  cosbx = cosb(x)
  sinbx = sinb(x)
dcosx = tape.gradient(cosx, x) 
dsinx = tape.gradient(sinx, x) 
dcosbx = tape.gradient(cosbx, x) 
dsinbx = tape.gradient(sinbx, x) 

def test(y1, y2, t = 0.0017):
  plt.plot(x, y1, linewidth=5)
  plt.plot(x, y2)
  err = np.max(np.abs(y1 - y2))
  print(err)
  assert err < t

test(cosx, cosbx)
test(dcosx, dcosbx)
test(sinx, sinbx)
test(dsinx, dsinbx)

runtimes = []
for _ in range(100):
  t0 = time.time()
  y = cosb(x)
  y = sinb(x)
  t1 = time.time()
  runtimes.append(t1 - t0)
time_approx = 1000 * np.median(runtimes)

runtimes = []
for _ in range(100):
  t0 = time.time()
  y = tf.math.cos(x)
  y = tf.math.sin(x)
  t1 = time.time()
  runtimes.append(t1 - t0)
time_default = 1000 * np.median(runtimes)
print(f'{time_default:0.5f} -> {time_approx:0.5f} | {time_default / time_approx:0.5f}x')